In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.max_rows', 1000)
#pd.set_option('display.width', 180)


In [8]:
hdf_file = '/opt/data36/daliu/realtime/spykshrk/timing/test_animal.rec_merged.h5'

store = pd.HDFStore(hdf_file, mode='r')

store['rec_3']